# SQLAlchemy
Is a library that facilitates the communication between Python programs and databases. SQLAlchemy provides a standard interface that allows developers to create database-agnostic code to communicate with a wide variety of database engines.

In [1]:
import sqlalchemy as db

from sqlalchemy import create_engine,inspect

import pandas as pd

In [2]:

engine = db.create_engine('sqlite:///test.sqlite') #Create test.sqlite automatically
connection = engine.connect()

# Holds a collection of Table objects
# The Table objects in the collection and their columns may participate in implicit SQL execution.
metadata = db.MetaData()

emp = db.Table('emp', metadata,
              db.Column('Id', db.Integer()),
              db.Column('name', db.String(255), nullable=False),
              db.Column('salary', db.Float(), default=100.0),
              db.Column('active', db.Boolean(), default=True)
              )

metadata.create_all(engine) #Creates the table

In [17]:
# metadata provides the basic and relevant information about the database objects.
employeeT = db.Table('emp',metadata)

# Print the column names
print(employeeT.columns.keys())

['Id', 'name', 'salary', 'active']


In [12]:
#this is used to look at the schema of elements in a database
# Get Database Information
inspector = inspect(engine)

# key is the name of the column and value is the attribute for that field
print(inspector.get_columns('emp'))

[{'name': 'Id', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'name', 'type': VARCHAR(length=255), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'salary', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'active', 'type': BOOLEAN(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}]


## Inserting Data using SQLAlchemy

In [13]:
# Inserting many records at ones

query = db.insert(emp) 
values_list = [{'Id':'2', 'name':'ram', 'salary':80000, 'active':False},
               {'Id':'3', 'name':'ramesh', 'salary':70000, 'active':True}]
ResultProxy = connection.execute(query,values_list)

In [14]:
# insert a row
engine.execute('INSERT INTO "emp" '
               '(id, name, salary, active) '
               'VALUES (4,"Sandy",40000,True)')


In [16]:
result = connection.execute('SELECT * FROM emp')
dfNew = pd.DataFrame(result)
dfNew.columns = results[0].keys()
dfNew.head()

,Id,name,salary,active
0,2,ram,80000.0,0
1,3,ramesh,70000.0,1
2,4,Sandy,40000.0,1


In [15]:
'''

results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df

'''

,Id,name,salary,active
0,2,ram,80000.0,False
1,3,ramesh,70000.0,True
2,4,Sandy,40000.0,True


## Selecting rows using SQLAlchemy sintax

In [25]:

#Equivalent to 'SELECT * FROM table_name'
query = db.select([emp])
ResultProxy = connection.execute(query)

# The ResultProxy object return information of rows
ResultSet = ResultProxy.fetchall()

df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df.head()


,Id,name,salary,active
0,2,ram,80000.0,False
1,3,ramesh,70000.0,True
2,4,Sandy,40000.0,True


In [26]:
# Get the number of Rows and columns
df.shape

(3, 4)

### Close de connection and release memory

In [3]:
ResultProxy.close()
connection.close()

NameError: name 'ResultProxy' is not defined

## Selecting records using Pandas

We can use the pandas read_sql_query function to read the results of a SQL query directly into a pandas DataFrame. The below code will execute the same query that we just did, but it will return a DataFrame. It has several advantages over the query we did above:

- It doesn’t require us to create a Cursor object or call fetchall at the end.
- It automatically reads in the names of the headers from the table.
- It creates a DataFrame, so we can quickly explore the data.

In [28]:
#get all the rows from the "test" table in the database
sql = "SELECT id, name, salary, active FROM emp;"

#the read_sql function takes in the SQL statement for the information requested and the engine (to connect to database)
data_df = pd.read_sql(sql, engine)
data_df

,Id,name,salary,active
0,2,ram,80000.0,0
1,3,ramesh,70000.0,1
2,4,Sandy,40000.0,1


## Open a Database using Pandas

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("flights.db")


In [2]:
### Names of tables in the database
dfTables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
dfTables

,name
0,airports
1,airlines
2,routes


## Selecting rows from an existing database

In [3]:
df = pd.read_sql_query("select *  from airlines;", conn)
df.head()

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [4]:
# Number of rows and columns 
df.shape


(6048, 9)

In [5]:
# Selecting only the columns we want to display
df = pd.read_sql_query("select id, name, alias, country, active  from airlines;", conn)
df.head()

,id,name,alias,country,active
0,1,Private flight,\N,None,Y
1,2,135 Airways,\N,United States,N
2,3,1Time Airline,\N,South Africa,Y
3,4,2 Sqn No 1 Elementary Flying Training School,\N,United Kingdom,N
4,5,213 Flight Unit,\N,Russia,N


In [6]:
 pd.read_sql_query("select id, name, alias, country, active  from airlines limit 3;", conn)

,id,name,alias,country,active
0,1,Private flight,\N,None,Y
1,2,135 Airways,\N,United States,N
2,3,1Time Airline,\N,South Africa,Y


## Inserting rows with Pandas

In [7]:
# We specify 9 values to insert, one for each column in airlines. This will add a new row to the table.
# Cursors allow us to execute SQL queries against a database:
cur = conn.cursor()
cur.execute("insert into airlines values (6048, 19846, 'Test flight', '', '', null, null, null, 'Y')")

- By default, sqlite3 opens a transaction when you do any query that modifies the database.
- SQLite doesn’t write to the database until you commit a transaction. A transaction consists of 1 or more queries that all make changes to the database at once. This is designed to make it easier to recover from accidental changes, or errors. Transactions allow you to run several queries, then finally alter the database with the results of all of them

In [8]:
conn.commit()

In [9]:
# Checking if the record was inserted
pd.read_sql_query("select id, name, alias, country, active  from airlines where id=19846;", conn)

,id,name,alias,country,active
0,19846,Test flight,,None,Y


## Inserting Rows passing parameters into query
- Most of the time, when you insert data into a database, it won’t be hardcoded, it will be dynamic values you want to pass in. These dynamic values might come from downloaded data, or might come from user input.

- When working with dynamic data, it might be tempting to insert values using Python string formatting

In [10]:
# Checking if the record exists
pd.read_sql_query("select id, name, country, active from airlines where id=19847;", conn)

,id,name,country,active


**Any ?** value in the query will be replaced by a value in values. The first **?** will be replaced by the first item in values, the second by the second, and so on. This works for any type of query.

In [11]:
cur = conn.cursor()
values = (6049, 19847,'Test Flight', 'Y')
cur.execute("insert into airlines values (?, ?, ?, '', '', null, null, null, ?)", values)
conn.commit()

In [12]:
# Checking if the record was inserted
pd.read_sql_query("select id, name, country, active from airlines where id=19847;", conn)

,id,name,country,active
0,19847,Test Flight,None,Y


## Updating rows
We can modify rows in a SQLite table using the execute method.

In [13]:
pd.read_sql_query("select  id, name, country, active from airlines where id=19847;", conn)

,id,name,country,active
0,19847,Test Flight,None,Y


In [14]:
cur = conn.cursor()
values = ('New Airlines','USA', 19847)
cur.execute("update airlines set name=?, country=? where id=?", values)
conn.commit()

In [15]:
pd.read_sql_query("select id, name, country, active from airlines where id=19847;", conn)

,id,name,country,active
0,19847,New Airlines,USA,Y


## Deleting rows
Finally, we can delete the rows in a database using the execute method

In [16]:
pd.read_sql_query("select id, name, country, active from airlines where id=19847;", conn)

,id,name,country,active
0,19847,New Airlines,USA,Y


In [17]:
cur = conn.cursor()
values = (19847,)
cur.execute("delete from airlines where id=?", values)
conn.commit()

In [18]:
pd.read_sql_query("select * from airlines where id=19847;", conn)

,index,id,name,alias,iata,icao,callsign,country,active


## Altering tables with Pandas

In [19]:
#  Adding a new column: airplanes field to the airlines table that indicates how many airplanes each airline owns
#  Cursors allow us to execute SQL queries against a database:
cur = conn.cursor()
cur.execute("alter table airlines add column airplanes integer;")

In [20]:
pd.read_sql_query("select * from airlines limit 1;", conn)

,index,id,name,alias,iata,icao,callsign,country,active,airplanes
0,0,1,Private flight,\N,-,None,None,None,Y,None


## Creating tables
We can create tables by executing a SQL query. We can create a table to represent each daily flight on a route, with the following columns:

- id — integer
- departure — date, when the flight left the airport
- arrival — date, when the flight arrived at the destination
- number — text, the flight number
- route_id — integer, the id of the route the flight was flying

In [21]:
cur = conn.cursor()
cur.execute("create table daily_flights (id integer, departure date, arrival date, number text, route_id integer)")
conn.commit()


In [22]:
# Once we create a table, we can insert data into it normally:

cur.execute("insert into daily_flights values (1, '2016-09-28 0:00', '2016-09-28 12:00', 'T1', 1)")
conn.commit()

In [23]:
# Select data from new table
pd.read_sql_query("select id, departure, arrival, number, route_id from daily_flights;", conn)

,id,departure,arrival,number,route_id
0,1,2016-09-28 0:00,2016-09-28 12:00,T1,1


In [24]:

conn.close()